# Agenda

1. Pandas, NumPy, and PyArrow
2. Arrow -- what it is
3. PyArrow for file loading/saving
4. PyArrow as a back-end data type
5. Transitioning to PyArrow

# Pandas, NumPy, and PyArrow

NumPy has been around for about 20 years. It gives us the speed and efficiency of C data types, but a Python API. That means we can use Python for numeric computing.

NumPy defines the "NumPy array," which contains a number of values all of the same type. These are all stored in C, which means that (in contrast with normal Python programming), we have to specify not only the type of values, but their lengths, as well.

In [1]:
import numpy as np

In [2]:
a = np.array([10, 20, 30, 40, 50])
a

array([10, 20, 30, 40, 50])

In [3]:
type(a)

numpy.ndarray

In [4]:
a.dtype

dtype('int64')

In [5]:
64 * 5

320

In [6]:
mylist = [10, 20, 30, 40, 50]
import sys
sys.getsizeof(mylist)

104

In [7]:
a = np.array([10, 20, 30, 40, 50], dtype='int8')
a

array([10, 20, 30, 40, 50], dtype=int8)

NumPy is great! But it's also a bit low level. About 15 years ago, Wes McKinney invented Pandas, which is a wrapper around NumPy, giving us a lot of convenience methods. NumPy handles n-dimensional arrays. Pandas has only two main data structures, one is the Series (1D) and also the DataFrame (2D).

Pandas gives us lots of methods for:

- Loading data from different formats
- Storing data in different formats
- Analyzing in many ways
- Cleaning our data
- Visualizing our data

When we store data in our Pandas series or data frame, it's being stored in a NumPy array behind the scenes:

- A series is actually a wrapper around a single NumPy array
- Each of a data frame's columns is a NumPy array, but there are very fast transformations that allow us to retrieve by row, as well as by column.

For many years, this has worked just fine... mostly. Where are the problems?

- Strings -- currently, Pandas stores text as Python strings, with pointers from NumPy arrays into Python memory
- There's no compression or general categorization/normalization of textual (or other) data
- Much of the analysis we want to do is columnar, and NumPy just isn't that fast at working in columns, even when we think of a data frame as containing multiple series (i.e., NumPy arrays)
- NumPy is great for Python, but it's not interoperable with other systems.

Wes and others said: Let's create a new data structure that'll work in Pandas and also in lots of other systems and langauges. That became Apache Arrow, an open-source project. It just released version 20 a month or so ago.

It supports many data structures (arrays, tables, dictionaries, etc.), with copy-on-write, compression, with reduction of duplications, and also nullable types. Python supports it through PyArrow, its bindings, but you also have Arrow compatibility with R, Apache Spark, and other languages/systems.



# PyArrow and Pandas

For a long time, you could convert a PyArrow Table to a Pandas data frame, and vice versa. They were interoperable, but needed translation.

A few years ago, several Pandas core developers decided it would be worth starting the journey of moving Pandas from NumPy to PyArrow as a back end.

The idea is that Pandas will continue to work as before, with all of its methods and functionality, but the back end will be PyArrow, rather than NumPy.

In a few years, we can expect that PyArrow will be the default back-end storage. I don't think that NumPy is going away, either in general or in Pandas. But Pandas 3.0 will require PyArrow to be installed.

# Loading/saving files

Already today, in a non-experimental way, you can use PyArrow in your Pandas code to read from CSV files (and write to them, too).

If you use `pd.read_csv`, then Pandas needs to do several things:

- Load the file into memory, often in multiple chunks, if it's too big
- Evaluate what dtype you're going to want for each column (if you don't specify), choosing between `int64`, `float64`, and `object` (i.e., string)
- Create the data frame based on that
- Pandas, of course, uses a single core, with a single process and a single thread

PyArrow uses the Arrow engine, which uses multiple cores, and multiple threads, to read from a file, which makes it far faster.

In [8]:
filename = '/Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv'

!ls -lh $filename

-rw-r--r-- 1 reuven staff 656M Jul 29  2021 /Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv


In [12]:
import time
import pandas as pd
from pandas import Series, DataFrame

In [13]:
start_time = time.perf_counter()
df = pd.read_csv(filename)
end_time = time.perf_counter()

total_time = end_time - start_time
print(f'Total time: {total_time:0.2f}')

Total time: 6.42


In [14]:
start_time = time.perf_counter()
df = pd.read_csv(filename, engine='pyarrow')
end_time = time.perf_counter()

total_time = end_time - start_time
print(f'Total time: {total_time:0.2f}')

Total time: 1.05


In [15]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols: 'UsecolsArgType' = None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters: 'Mapping[Hashable, Callable] | None' = None,
    true_values: 'list | None' = None,
    false_values: 'list | None' = None,
    skipinitialspace: 'bool' = False,
    skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values: 'Hashable | Iterable[Hashable] | Mapping[Hashable, Iterable[Hashable]] | None' = None,
  

In [16]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [17]:
df.dtypes

VendorID                         int64
tpep_pickup_datetime     datetime64[s]
tpep_dropoff_datetime    datetime64[s]
passenger_count                  int64
trip_distance                  float64
RatecodeID                       int64
store_and_fwd_flag              object
PULocationID                     int64
DOLocationID                     int64
payment_type                     int64
fare_amount                    float64
extra                          float64
mta_tax                        float64
tip_amount                     float64
tolls_amount                   float64
improvement_surcharge          float64
total_amount                   float64
congestion_surcharge           float64
dtype: object

When the Arrow team put together the specification, they also specified two file formats that can be used for reading/writing data. These are binary formats, which means that they are faster to load, but they also represent the data using precise dtypes, such as `int8` or `datetime`.

These formats are:

- `feather`, which is less compressed (and thus bigger on disk), but faster to read/write
- `parquet`, which is more compressed (and thus smaller on disk), but slower to read/write

In [21]:
%timeit df.to_feather('/tmp/mydata.feather')

498 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit df.to_parquet('/tmp/mydata.parquet')

1.54 s ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
ls -lh $filename

-rw-r--r-- 1 reuven 656M Jul 29  2021 /Users/reuven/Courses/Current/Data/nyc_taxi_2019-01.csv


In [23]:
ls -lh /tmp/mydata.*

-rw-r--r-- 1 reuven 254M May 20 20:42 /tmp/mydata.feather
-rw-r--r-- 1 reuven 126M May 20 20:42 /tmp/mydata.parquet


In [24]:
%timeit pd.read_feather('/tmp/mydata.feather')

265 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit pd.read_parquet('/tmp/mydata.parquet')

207 ms ± 25.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
